In [ ]:
import cv2
import os

# Specify the directory to save the images
save_dir = 'Dataset/Testing/2'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Load the pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open a connection to the webcam
cap = cv2.VideoCapture(0)

# Counter for saved images
image_counter = 0

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Check if the space button is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):  # Space button pressed
        for (x, y, w, h) in faces:
            # Extract the face ROI
            face_roi = frame[y:y+h, x:x+w]

            # Save the face ROI as an image
            image_path = os.path.join(save_dir, f'face_{image_counter}.png')
            cv2.imwrite(image_path, face_roi)
            print(f'Saved: {image_path}')
            image_counter += 1

    # Break the loop if 'q' is pressed
    if key == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
train_data_dir = './Dataset/Training'
test_data_dir = './Dataset/Testing'

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# image_size -->
img_size = (128, 128)
batch_size=32
# intializing
train_datagen = ImageDataGenerator(
rescale= 1./255
)



In [ ]:

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = img_size,
    batch_size=32,
    class_mode = 'categorical'
)

In [ ]:
num_classes = len(train_generator.class_indices)
print(num_classes)

In [ ]:
input_shape = (128, 128, 3)

In [ ]:
model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

#         layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

#         layers.BatchNormalization(),
#         layers.Dropout(0.25),

        # Second Convolutional Block
#         layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2, 2)),
#         layers.Dropout(0.25),

        # Third Convolutional Block
#         layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2, 2)),
#         layers.Dropout(0.25),

        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dense(16, activation='relu'),
#         layers.BatchNormalization(),
#         layers.Dropout(0.5),
        layers.Dense(32, activation='relu'),
#         layers.BatchNormalization(),
#         layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

from PIL import Image
model.fit(train_generator, epochs=7)

In [ ]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size = img_size,
    batch_size=32,
    class_mode = 'categorical'
)

In [ ]:
accuracy = model.evaluate(test_generator)


In [ ]:
y_pred = model.predict(test_generator)
print(y_pred)

In [ ]:
from tensorflow.keras.models import load_model

model.save('model.h5')  

In [ ]:
import cv2
import numpy as np
import tflite_runtime.interpreter as tflite
import RPi.GPIO as GPIO
import time
import sys

# GPIO setup
servo_pin = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(servo_pin, GPIO.OUT)
servo = GPIO.PWM(servo_pin, 50)
servo.start(0)

# Move servo to 0 degrees
servo.ChangeDutyCycle(2.5)
time.sleep(1)
servo.ChangeDutyCycle(0)

# Load the pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the TensorFlow Lite model
interpreter = tflite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Open a connection to the webcam
cap = cv2.VideoCapture(0)

# Define the target size for the model input
target_size = (128, 128)

# Assume we have the class names (update this based on your actual class names)
class_names = ['Skylar', 'Johnny', 'Tim']  # Replace with your actual class names

try:
    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Extract the face ROI
            face_roi = frame[y:y+h, x:x+w]
            cv2.imwrite("Image.png", face_roi)

            # Preprocess the face image for the model
            face_image = cv2.resize(face_roi, target_size)
            face_image = face_image.astype(np.float32)
            face_image = np.expand_dims(face_image, axis=0)
            face_image = face_image / 255.0  # Normalize pixel values

            # Make prediction
            interpreter.set_tensor(input_details[0]['index'], face_image)
            interpreter.invoke()
            prediction = interpreter.get_tensor(output_details[0]['index'])
            predicted_class = class_names[np.argmax(prediction)]
            confidence = np.max(prediction)

            if predicted_class != "Skylar":
                print(f"Detected {predicted_class}!")
                servo.ChangeDutyCycle(7.5)  # Move to 90 degrees
                time.sleep(1)
                servo.ChangeDutyCycle(0)
                raise SystemExit

            # Display the prediction on the frame
            text = f'{predicted_class}: {confidence:.2f}'
            cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('Face Recognition', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except SystemExit:
    print("Exiting program")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Release the webcam and close all windows
    cap.release()
    cv2.destroyAllWindows()
    servo.stop()
    GPIO.cleanup()

In [ ]:
import tensorflow as tf
print(tf.__version__)